In [0]:
#fields of email df
import pandas as pd
import csv

#set df for email data
email_df_col=['date','user','size','attachment_count','to_domain','cc_domain','bcc_domain',
              'from_domain','leak_risk','thief_risk','sabotage_risk']
#email_df= pd.DataFrame( columns=email_df_col)
#load classified domains
domains_dim_df=pd.read_csv(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\all_users_domains_classified.csv')
leak_domains=  set(domains_dim_df.loc[:,'domain'][domains_dim_df['type']=='leaks'])
thief_domains=set(domains_dim_df.loc[:,'domain'][(domains_dim_df['competitor'] == 1) | (domains_dim_df['type'] == 'jobsearch')])
sabotage_domains=set(domains_dim_df.loc[:,'domain'][domains_dim_df['type'].isin (['spyware','downloads','hacking'])])


In [0]:
email_file = open(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\email.csv', "r")
email_file_out = open(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\email_out.csv', "w")


with email_file_out:
    writer=csv.writer(email_file_out)
    writer.writerow(email_df_col)
    email_file.readline()
    for line in email_file:  
        leak_risk=False
        thief_risk=False
        sabotage_risk=False
        fields=line.split(",")
        to_domain=''
        cc_domain=''
        bcc_domain=''
        from_domain=''
        for idx in range(4,8): 
            try:               
                emails  =fields[idx].split(';')
                idx_email=0
                for email in emails:
                    dom_email=email.split('@')[1]
                    if idx_email ==0:
                        if idx == 4:
                            to_domain= dom_email
                        elif idx == 5:
                            cc_domain= dom_email
                        elif idx == 6:
                            bcc_domain= dom_emaiil
                        else:
                            from_domain= dom_email
                    else:
                        idx_email += 1        
                leak_risk |= dom_email in leak_domains
                thief_risk |= dom_email in thief_domains
                sabotage_risk |= dom_email in sabotage_domains
            except:
                 continue
        email_row=[fields[1],fields[2],fields[8], fields[9],to_domain, cc_domain,
                 bcc_domain, from_domain, 1 if leak_risk else 0,
                   1 if thief_risk else 0, 1 if sabotage_risk else 0] 
        writer.writerow(email_row)
        #email_df=email_df.append(pd.Series(email_row, index=email_df_col), ignore_index=True)       
email_file.close() 
email_file_out.close()

In [0]:
import pandas as pd
email= pd.read_csv(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\email_out.csv', parse_dates=['date'])

In [0]:
email=pd.merge(email, domains_dim_df[['domain','type','competitor']],left_on='to_domain',right_on='domain', how='left')

In [0]:
email['day_of_week']=email['date'].apply( lambda x: x.weekday()) 
email['hour'] = email['date'].apply( lambda x: x.hour*3600+ x.minute*60 +x.second) 

In [0]:
email

,date,user,size,attachment_count,to_domain,cc_domain,bcc_domain,from_domain,leak_risk,thief_risk,sabotage_risk,domain,type,competitor,day_of_week,hour
0,2010-01-02 07:11:45,LAP0338,25830,0,lockheedmartin.com,dtaa.com,NaN,dtaa.com,0,1,0,lockheedmartin.com,planes,1.0,5,25905
1,2010-01-02 07:12:16,MOH0273,29942,0,bellsouth.net,NaN,NaN,optonline.net,0,0,0,bellsouth.net,news,0.0,5,25936
2,2010-01-02 07:13:00,LAP0338,28780,0,netzero.com,NaN,NaN,earthlink.net,0,0,0,netzero.com,webmail,0.0,5,25980
3,2010-01-02 07:13:17,LAP0338,21907,0,comcast.net,NaN,NaN,earthlink.net,0,0,0,comcast.net,news,0.0,5,25997
4,2010-01-02 07:13:28,MOH0273,17319,0,juno.com,NaN,NaN,optonline.net,0,0,0,juno.com,webmail,0.0,5,26008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2629974,2011-05-16 20:54:18,HRL0540,33088,0,dtaa.com,dtaa.com,NaN,dtaa.com,0,0,0,NaN,NaN,NaN,0,75258
2629975,2011-05-16 20:54:43,LAF0991,33249,1,dtaa.com,dtaa.com,NaN,dtaa.com,0,0,0,NaN,NaN,NaN,0,75283
2629976,2011-05-16 21:08:12,LAF0991,41336,1,charter.net,NaN,NaN,sbcglobal.net,0,0,0,charter.net,isp,0.0,0,76092
2629977,2011-05-16 21:15:35,JMW0638,39908,0,dtaa.com,NaN,NaN,dtaa.com,0,0,0,NaN,NaN,NaN,0,76535


In [0]:
def get_risk_out_of_work(row):  
    try:     
        week_end_leak=week_end_thief=week_end_sabotage=0
        off_hour_leak=off_hour_leak=off_hour_leak=0
        if (row.day_of_week >4):            
            week_end_leak= 1 if row.leak_risk else 0
            week_end_thief= 1 if row.thief_risk else 0
            week_end_sabotage= 1 if row.sabotage_risk else 0
        if ( (row.hour < 28800) | (row.hour >61200)):            
            off_hour_leak= 1 if row.leak_risk else 0
            off_hour_thief= 1 if row.thief_risk else 0
            off_hour_sabotage= 1 if row.sabotage_risk else 0
        return week_end_leak,week_end_thief,week_end_sabotage,off_hour_leak,off_hour_leak,off_hour_leak
    except:
        return 0,0,0,0,0,0    

email[['week_end_leak','week_end_thief','week_end_sabotage',
         'off_hour_leak','off_hour_thief','off_hour_sabotage']]= email.apply(lambda x: get_risk_out_of_work(x),axis=1 ,result_type='expand')


In [0]:
email

,date,user,size,attachment_count,to_domain,cc_domain,bcc_domain,from_domain,leak_risk,thief_risk,...,competitor,day_of_week,hour,week_end_leak,week_end_thief,week_end_sabotage,off_hour_leak,off_hour_thief,off_hour_sabotage,period
0,2010-01-02 07:11:45,LAP0338,25830,0,lockheedmartin.com,dtaa.com,NaN,dtaa.com,0,1,...,1.0,5,25905,0,1,0,0,0,0,201001
1,2010-01-02 07:12:16,MOH0273,29942,0,bellsouth.net,NaN,NaN,optonline.net,0,0,...,0.0,5,25936,0,0,0,0,0,0,201001
2,2010-01-02 07:13:00,LAP0338,28780,0,netzero.com,NaN,NaN,earthlink.net,0,0,...,0.0,5,25980,0,0,0,0,0,0,201001
3,2010-01-02 07:13:17,LAP0338,21907,0,comcast.net,NaN,NaN,earthlink.net,0,0,...,0.0,5,25997,0,0,0,0,0,0,201001
4,2010-01-02 07:13:28,MOH0273,17319,0,juno.com,NaN,NaN,optonline.net,0,0,...,0.0,5,26008,0,0,0,0,0,0,201001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2629974,2011-05-16 20:54:18,HRL0540,33088,0,dtaa.com,dtaa.com,NaN,dtaa.com,0,0,...,NaN,0,75258,0,0,0,0,0,0,201105
2629975,2011-05-16 20:54:43,LAF0991,33249,1,dtaa.com,dtaa.com,NaN,dtaa.com,0,0,...,NaN,0,75283,0,0,0,0,0,0,201105
2629976,2011-05-16 21:08:12,LAF0991,41336,1,charter.net,NaN,NaN,sbcglobal.net,0,0,...,0.0,0,76092,0,0,0,0,0,0,201105
2629977,2011-05-16 21:15:35,JMW0638,39908,0,dtaa.com,NaN,NaN,dtaa.com,0,0,...,NaN,0,76535,0,0,0,0,0,0,201105


In [0]:
email_monthly_activity=email.groupby(['user',pd.Grouper(key='date',freq="M")]).agg(
    leak_risk_cnt= ('leak_risk', "sum") ,
    leak_we_cnt= ('week_end_leak', "sum") ,
    leak_off_hr_cnt=('off_hour_leak', "sum") ,
    thief_risk_cnt=('thief_risk', "sum"),  
    thief_we_cnt= ('week_end_thief', "sum") ,
    thief_off_hr_cnt=('off_hour_thief', "sum") ,    
    sabotage_risk_cnt=('sabotage_risk', "sum"),
    sabotage_we_cnt=('week_end_sabotage', "sum"),
    sabotage_off_hr_cnt=('off_hour_sabotage', "sum"),     
    competitor_cnt=('competitor', "sum")
)

email_monthly_activity=email_monthly_activity.reset_index()
email_monthly_activity['period']=email_monthly_activity['date'].dt.strftime('%Y%m')

In [0]:
email_monthly_activity=email_monthly_activity[['period','user','leak_risk_cnt','leak_we_cnt','leak_off_hr_cnt','thief_risk_cnt','thief_we_cnt','thief_off_hr_cnt','sabotage_risk_cnt','sabotage_we_cnt','sabotage_off_hr_cnt','competitor_cnt']]

In [0]:
email['period']=email['date'].dt.strftime('%Y%m')
email_pivot=email[['user','period','type','domain']].pivot_table( index=['user','period',], columns='type' ,aggfunc='count', fill_value=0)
email_pivot=email_pivot.reset_index()
email_pivot.columns= list (email_pivot.columns.get_level_values(0)[:2])+list( email_pivot.columns.get_level_values(1)[2:])
email_pivot

,user,period,isp,news,planes,searchengines,shopping,sports,webmail
0,AAE0190,201001,22,19,4,6,11,13,36
1,AAE0190,201002,22,18,8,13,5,16,40
2,AAE0190,201003,15,22,11,9,13,9,31
3,AAE0190,201004,17,18,11,11,8,5,35
4,AAE0190,201005,15,21,16,13,9,10,45
...,...,...,...,...,...,...,...,...,...
15893,ZSL0305,201101,3,4,2,2,0,1,3
15894,ZSL0305,201102,2,4,1,0,0,1,2
15895,ZSL0305,201103,3,5,2,1,0,0,4
15896,ZSL0305,201104,1,3,1,1,0,0,3


In [0]:
final_email_risk=pd.merge(email_monthly_activity,email_pivot ,on=['user','period'])

In [0]:
final_email_risk.to_csv(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\final_email_risk.csv', index=False)

In [0]:
final_email_risk

,period,user,leak_risk_cnt,leak_we_cnt,leak_off_hr_cnt,thief_risk_cnt,thief_we_cnt,thief_off_hr_cnt,sabotage_risk_cnt,sabotage_we_cnt,sabotage_off_hr_cnt,competitor_cnt,isp,news,planes,searchengines,shopping,sports,webmail
0,201001,AAE0190,0,0,0,21,0,0,0,0,0,15.0,22,19,4,6,11,13,36
1,201002,AAE0190,0,0,0,20,0,0,0,0,0,13.0,22,18,8,13,5,16,40
2,201003,AAE0190,0,0,0,40,0,0,0,0,0,24.0,15,22,11,9,13,9,31
3,201004,AAE0190,0,0,0,30,0,0,0,0,0,19.0,17,18,11,11,8,5,35
4,201005,AAE0190,0,0,0,31,0,0,0,0,0,25.0,15,21,16,13,9,10,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15893,201101,ZSL0305,0,0,0,2,0,0,0,0,0,2.0,3,4,2,2,0,1,3
15894,201102,ZSL0305,0,0,0,2,0,0,0,0,0,1.0,2,4,1,0,0,1,2
15895,201103,ZSL0305,0,0,0,2,0,0,0,0,0,2.0,3,5,2,1,0,0,4
15896,201104,ZSL0305,0,0,0,1,0,0,0,0,0,1.0,1,3,1,1,0,0,3


In [0]:
final_http_risk=pd.read_csv(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\final_http_risk.csv')
final_http_risk['period']=final_http_risk['period'].astype(str)

In [0]:
final_risk_insider=pd.concat([final_http_risk, final_email_risk]).groupby(['user','period']).sum().reset_index()


In [0]:
final_risk_insider

,user,period,leak_risk_cnt,leak_we_cnt,leak_off_hr_cnt,thief_risk_cnt,thief_we_cnt,thief_off_hr_cnt,sabotage_risk_cnt,sabotage_we_cnt,...,travel,updatesites,urlshortener,warez,webmail,webphone,webradio,webtv,wellness,competitor_cnt
0,AAE0190,201001,0,0,0,45,0,0,161,0,...,25.0,0.0,0.0,28.0,36,3.0,1.0,389.0,0.0,15.0
1,AAE0190,201002,0,0,0,25,0,0,200,0,...,38.0,0.0,0.0,32.0,41,4.0,1.0,447.0,0.0,13.0
2,AAE0190,201003,0,0,0,68,0,0,235,0,...,34.0,0.0,0.0,36.0,31,12.0,1.0,560.0,0.0,24.0
3,AAE0190,201004,0,0,0,50,0,0,222,0,...,48.0,0.0,0.0,29.0,36,8.0,3.0,511.0,0.0,19.0
4,AAE0190,201005,0,0,0,44,0,0,186,0,...,34.0,5.0,0.0,24.0,46,1.0,7.0,371.0,0.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15894,ZSL0305,201101,0,0,0,20,0,0,8,0,...,1.0,0.0,0.0,8.0,3,3.0,0.0,2.0,0.0,2.0
15895,ZSL0305,201102,0,0,0,26,0,0,7,0,...,0.0,0.0,0.0,9.0,2,3.0,0.0,1.0,0.0,1.0
15896,ZSL0305,201103,0,0,0,12,0,0,7,0,...,0.0,0.0,2.0,7.0,4,3.0,0.0,0.0,0.0,2.0
15897,ZSL0305,201104,0,0,0,16,0,0,10,0,...,1.0,0.0,0.0,9.0,3,2.0,0.0,1.0,0.0,1.0


In [0]:
final_risk_insider.to_csv(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\final_insdier_risk.csv', index=False)